<a href="https://colab.research.google.com/github/yinon2592/DL_Project_046211/blob/yinon_branch/section_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!mkdir -p data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/investigating-sentiment-analysis/data/training.1600000.processed.noemoticon.csv.zip -P data
!unzip -n -d data data/training.1600000.processed.noemoticon.csv.zip
!pip install transformers

--2023-06-23 11:06:49--  https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/investigating-sentiment-analysis/data/training.1600000.processed.noemoticon.csv.zip
Resolving nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)... 162.243.189.2
Connecting to nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)|162.243.189.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85088192 (81M) [application/zip]
Saving to: ‘data/training.1600000.processed.noemoticon.csv.zip’

training.1600000.pr 100%[===================>]  81.15M  15.9MB/s    in 6.2s    

2023-06-23 11:06:57 (13.0 MB/s) - ‘data/training.1600000.processed.noemoticon.csv.zip’ saved [85088192/85088192]

Archive:  data/training.1600000.processed.noemoticon.csv.zip
  inflating: data/training.1600000.processed.noemoticon.csv  
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.0 MB/s eta 0:00:

In [3]:
import pandas as pd
import re
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
# Step 1: Dataset Preparation
file_path="data/training.1600000.processed.noemoticon.csv"
df = pd.read_csv(file_path, encoding='ISO-8859-1', header=None)
df = df[[0, 5]]
df.columns = ['label', 'text']

df = df.sample(100, random_state=1)

# # df['label'] = df['label'].replace({0: 'negative', 2: 'neutral', 4: 'positive'})
# df['label'] = df['label'].replace({0: 'negative', 2: 'neutral', 4: 1})
# # Drop the rows with 'neutral' sentiment
# df = df[df['label'] != 'neutral']
df['label'] = df['label'].replace({4: 1})
df = df[df['label'] != 2]
print(df.label.value_counts())
print(df.sample(16))

# Step 2: Data Preprocessing
def clean_text(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Remove usernames
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)  # Remove URLs
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text)  # Remove special characters
    # Remove newlines and multiple whitespaces
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    # Remove special characters and punctuations
    text = re.sub(r'[^\w\s]', '', text)

    text = text.lower().strip()
    return text

df['text'] = df['text'].apply(clean_text)



0    57
1    43
Name: label, dtype: int64
         label                                               text
1360045      1  Just got done distributing postcards for our $...
654427       0  Think someone has sharted themselves on this t...
657012       0               @supercoolkp In Oxford that month.  
1038253      1                      @she_shines92 fangirl squee! 
1035799      1  I do not now nor ever will, trust the structur...
129162       0                         I feel neglected....  #sad
150953       0  is not going to the gym today  My shoulder hur...
1413143      1        http://twitpic.com/6rg4l All in for repair 
514293       0  i miss nikki nu nu already  shes always there ...
1299511      1  Wow... Thunder too.  My Grandpa used to tell m...
1037777      1     @ddlovatofans101 oh k good luck with that  lol
1533289      1  Checking to see who may have Saude Creek wine ...
832114       1  @kateweb Haha, I know - what a ridicukous scen...
275031       0  is loving xna. The

In [46]:
# Convert the 'text' column to a list
text_data = df['text'].tolist()
labels = torch.tensor(df['label'].tolist())

# Step 3: Tokenization
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
print(tokenizer)
# encoded_data = tokenizer(
#     text_data,
#     truncation=True,
#     padding=True,
#     max_length=128,
#     return_tensors='pt',
#     add_special_tokens=True
# )


# Batch tokenization
encoded_data = tokenizer.batch_encode_plus(
    text_data,
    truncation=True,
    padding='max_length',
    max_length=1024,
    return_tensors='pt',
    add_special_tokens=True
)


# Prepare the encoded data for feature extraction
input_ids = encoded_data['input_ids']
attention_masks = encoded_data['attention_mask']


# max_length = max(len(indices) for indices in input_ids)
# print(max_length)

# Convert the input_ids and attention_masks to tensors
input_ids = input_ids.squeeze()
attention_masks = attention_masks.squeeze()

# Print the shape of input_ids and attention_masks
print(encoded_data)
print("input_ids shape:", input_ids.shape)
print("attention_masks shape:", attention_masks.shape)

GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True)
{'input_ids': tensor([[   72,  2051,   299,  ..., 50257, 50257, 50257],
        [  568,  1312,   550,  ..., 50257, 50257, 50257],
        [ 1219,    71,  3595,  ..., 50257, 50257, 50257],
        ...,
        [18108,   257,  1049,  ..., 50257, 50257, 50257],
        [   72,  1254, 24007,  ..., 50257, 50257, 50257],
        [ 4919,   460,   356,  ..., 50257, 50257, 50257]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
       

In [47]:
from torch.utils.data import TensorDataset, DataLoader
# Convert the input_ids, attention_masks, and labels to TensorDataset
dataset = TensorDataset(input_ids, attention_masks, labels)
# dataset = TensorDataset(encoded_data, labels)

print(dataset)

# Define batch size
batch_size = 1

# Create data loader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
# def custom_collate_fn(batch):
#     # Modify the batch dimensions as per your requirement
#     input_ids = torch.stack([item[0] for item in batch], dim=1)  # Stack along the second dimension
#     attention_masks = torch.stack([item[1] for item in batch], dim=1)  # Stack along the second dimension
#     labels = torch.tensor([item[2] for item in batch])

#     return input_ids, attention_masks, labels

# # Create data loader with custom collate function
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)


In [48]:
from transformers import GPT2Model, GPT2Tokenizer
import torch.nn as nn

# Load the pre-trained GPT-2 model
gpt2_model = GPT2Model.from_pretrained('gpt2')

# Define the classification layer
num_labels = 2  # Number of sentiment classes (positive, negative)
classification_layer = nn.Sequential(
    nn.Linear(gpt2_model.config.hidden_size, num_labels),
    nn.Sigmoid()
)

# Print the model architecture
print(gpt2_model)
print(classification_layer)

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)
Sequential(
  (0): Linear(in_features=768, out_features=2, bias=True)
  (1): Sigmoid()
)


In [54]:
import torch.optim as optim

# Freeze the parameters of the pre-trained GPT-2 model
for param in gpt2_model.parameters():
    param.requires_grad = False

learning_rate = 0.001

# Define the optimizer and pass only the parameters of the last layer
optimizer = optim.Adam(classification_layer.parameters(), lr=learning_rate)

num_epochs = 5
criterion = torch.nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available
gpt2_model.to(device)
classification_layer.to(device)

# def batchify(data, bsz, seq_len=128):
#     """Divides the data into bsz separate sequences, removing extra elements
#     that wouldn't cleanly fit.

#     Args:
#         data: Tensor, shape [N]
#         bsz: int, batch size

#     Returns:
#         Tensor of shape [N // bsz, bsz]
#     """
#     tokenizer.model_max_length
#     # seq_len = data.size(0) // bsz
#     print(seq_len)
#     data = data[:seq_len * bsz]
#     data = data.view(bsz, seq_len).t().contiguous()
    # return data.to(device)


# Training loop
for epoch in range(num_epochs):
    gpt2_model.train()  # Set the GPT-2 model to training mode
    classification_layer.train()  # Set the classification layer to training mode

    for batch in dataloader:
        # print(batch)
        # print(batch[0].shape)
        # Clear the gradients of the optimizer
        optimizer.zero_grad()
        print(batch[0])
        # batch = {k: v.to(device) for k, v in batch.items()}
        batch_input = {"input_ids": batch[0]}
        # Forward pass through the classification layer
        # inputs = batch[0].to(device)
        # print(inputs.shape)
        # attention_masks = batch['attention_masks'].to(device)

        # inputs =  batchify(batch[0], batch_size)
        # attention_masks = batch[1].to(device)
        # print(attention_masks.shape)
        # attention_masks = batchify(batch[1], batch_size)
        # Make sure attention_masks has the shape [batch_size, sequence_length]
        # attention_masks = attention_masks.view(inputs.size(0), -1)

        # outputs = classification_layer(gpt2_model(inputs))
        # outputs = gpt2_model(input_ids=inputs, attention_mask=attention_masks)
        print(batch_input)
        outputs = gpt2_model(**batch_input)

        # Compute the loss and perform backpropagation
        labels = batch[2].to(device)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Optionally, calculate additional evaluation metrics

    # Print the epoch number and loss
    print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')


tensor([[   73,   888,   285,  ..., 50257, 50257, 50257]])
{'input_ids': tensor([[   73,   888,   285,  ..., 50257, 50257, 50257]])}


IndexError: ignored

In [ ]:
import torch.optim as optim

# Freeze the parameters of the pre-trained GPT-2 model
for param in gpt2_model.parameters():
    param.requires_grad = False

# Define the optimizer and pass only the parameters of the last layer
optimizer = optim.Adam(classification_layer[-1].parameters(), lr=learning_rate)

In [4]:
# Step 3: Split into Train, Validation, and Test Sets
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer
import textwrap

copy_df = df.head(5).copy()

def tokenize_text(text):
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2', truncation=True, padding=True, max_length=3)
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    # tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer.encode(text)

copy_df['text'] = copy_df['text'].apply(tokenize_text)

# wrapped_text = textwrap.fill(copy_df.sample(1).iloc(0), width=80)
print(copy_df.head(2)['text'])
# print(wrapped_text)

train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Split into train, validation, and test sets
train_df, test_df = train_test_split(df, test_size=test_ratio, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=val_ratio/(train_ratio+val_ratio), random_state=42)

# print("Train set size:", len(train_df))
# print("Validation set size:", len(val_df))
# print("Test set size:", len(test_df))

514293    [72, 2051, 299, 1134, 4106, 14364, 14364, 1541...
142282    [568, 1312, 550, 257, 4320, 938, 1755, 1312, 3...
Name: text, dtype: object


In [5]:
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer, AdamW
import torch

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2ForSequenceClassification.from_pretrained('gpt2')

# Add a classification layer on top
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available
model.to(device)

# Freeze initial layers
for param in model.parameters():
    param.requires_grad = False

# Optimizer and loss function
optimizer = AdamW(model.parameters())
loss_fn = torch.nn.CrossEntropyLoss()

from torch.utils.data import DataLoader, Dataset

# # Step 3: Tokenization and Dataset Creation
# class SentimentDataset(Dataset):
#     def __init__(self, data):
#         self.data = data
#         self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
#         self.tokenizer.add_special_tokens({'pad_token': '[PAD]'})

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         text = self.data.iloc[idx]['text']
#         label = self.data.iloc[idx]['label']
#         inputs = self.tokenizer.encode_plus(
#             text,
#             add_special_tokens=True,
#             max_length=128,  # Set your desired maximum sequence length
#             padding='max_length',
#             truncation=True,
#             return_tensors='pt'
#         )
#         return {
#             'input_ids': inputs['input_ids'].squeeze(),
#             'attention_mask': inputs['attention_mask'].squeeze(),
#             'labels': torch.tensor(1 if label == 'positive' else 0)  # Convert label to binary format
#         }




# # Create the datasets
# train_dataset = SentimentDataset(train_df)
# val_dataset = SentimentDataset(val_df)
# test_dataset = SentimentDataset(test_df)


# Define batch size and create the DataLoader
batch_size = 8  # Set your desired batch size
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)



num_epochs = 5

# Fine-tuning loop
for epoch in range(num_epochs):
  model.train()
  for batch in train_dataloader:
      # optimizer.zero_grad()
      # input_ids = [item['input_ids'] for item in batch]
      # attention_masks = [item['attention_mask'] for item in batch]
      # labels = torch.tensor([item['labels'] for item in batch]).to(device)

      # inputs = tokenizer.batch_encode_plus(
      #     input_ids,
      #     max_length=128,
      #     padding='longest',
      #     truncation=True,
      #     return_tensors='pt'
      # )
      # input_ids = inputs['input_ids'].to(device)
      # attention_masks = inputs['attention_mask'].to(device)

      # outputs = model(input_ids=input_ids, attention_mask=attention_masks, labels=labels)

      batch = {k: v.to(device) for k, v in batch.items()}
      outputs = model(**batch)

      loss = outputs.loss
      loss.backward()
      optimizer.step()

  # # Validation and early stopping logic
  # model.eval()
  # # Perform validation and monitor performance
  # # Implement early stopping if needed

KeyboardInterrupt: ignored